<a href="https://colab.research.google.com/github/apschlissel/w266-final-project/blob/main/BERT_reddit_baseline_(Kevin).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !config --NotebookApp.iopub_data_rate_limit=1.0e10
# # Source: https://stackoverflow.com/questions/50689784/in-google-collab-i-get-iopub-data-rate-exceeded

In [2]:
!pip install -q transformers

In [3]:
!pip install simpletransformers

In [4]:
# Useful Links / Source Code
# Source: https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c
# Source: https://medium.com/@senthilnathangautham/colab-gcp-compute-how-to-link-them-together-98747e8d940e
# Source: https://stackoverflow.com/questions/49031798/when-i-use-google-colaboratory-how-to-save-image-weights-in-my-google-drive

In [5]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline
from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
import logging
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import json
# Pull reddit data from reddit api
import requests

In [6]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

In [8]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('pigKA_TKnDkXcatEGcbo8g', 'nawGKK2MfPtC6vKz8TjaNEnmYfAggA')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'Katsuuu100',
        'password': 'Testing159753'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [9]:
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

# Pull from 5 classes. 5 classes = 5 subreddits.

# url_list = ["https://oauth.reddit.com/r/wallstreetbets/new",
            # "https://oauth.reddit.com/r/teenagers/new",
            # "https://oauth.reddit.com/r/copypasta/new",
              # "https://oauth.reddit.com/r/genz/new",
              # "https://oauth.reddit.com/r/unpopularopinion/new"]

my_list_of_dictionaries = []
total = 0


for i in range(5):
  page_count = 25
  url_list_check = [f"https://oauth.reddit.com/r/wallstreetbets/new/",
          f"https://oauth.reddit.com/r/teenagers/new/",
          f"https://oauth.reddit.com/r/copypasta/new/",
          f"https://oauth.reddit.com/r/genz/new/",
          f"https://oauth.reddit.com/r/unpopularopinion/new/",
          ]
  
  #print(url_list_check[i])
  if "wallstreetbets" in url_list_check[i]:
    name = "t3_tjoebd"
  elif "teenagers" in url_list_check[i]:
    name = "t3_tjt2rv"
  elif "copypasta" in url_list_check[i]:
    name = "t3_tjpvmw"
  elif "genz" in url_list_check[i]:
    name = "t3_ti9ds3"
  elif "unpopularopinion" in url_list_check[i]:
    name = "t3_tjpqel"
  for j in range(4):

    url_list = [f"https://oauth.reddit.com/r/wallstreetbets/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/teenagers/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/copypasta/new/?count={page_count}",
            f"https://oauth.reddit.com/r/genz/new/?count={page_count}",
            f"https://oauth.reddit.com/r/unpopularopinion/new/?count={page_count}",
            ]
    print(page_count)
    print(url_list[i])
    res = requests.get(url_list[i],
                    headers=headers)
                    # params={"limit": "100"})

    #print(json.dumps(res.json(), indent=4, sort_keys=False))

    reddit_dictionary = res.json()
    # print(json.dumps(reddit_dictionary["data"]["children"][0], indent=4, sort_keys=False))
    # print(len(reddit_dictionary["data"]["children"]))
    for k in range(len(reddit_dictionary["data"]["children"])):
      my_dictionary = {}
      my_dictionary["subreddit"] = reddit_dictionary["data"]["children"][k]["data"]["subreddit"]
      my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["selftext"]
      my_dictionary["title"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      my_dictionary["url"] = reddit_dictionary["data"]["children"][k]["data"]["url"]
      # print(reddit_dictionary["data"]["children"][k]["data"]["subreddit"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["selftext"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["url"])
      my_list_of_dictionaries.append(my_dictionary)
      total += 1
      name = reddit_dictionary["data"]["children"][k]["data"]["name"]
    # print(json.dumps(my_list_of_dictionaries, indent=4, sort_keys=False))
    print(name)
    page_count += 25

print("Total gathered:", total)

25
https://oauth.reddit.com/r/wallstreetbets/new/?count=25&after=t3_tjoebd
t3_tj5sx0
50
https://oauth.reddit.com/r/wallstreetbets/new/?count=50&after=t3_tj5sx0
t3_tia7aj
75
https://oauth.reddit.com/r/wallstreetbets/new/?count=75&after=t3_tia7aj
t3_ti0b84
100
https://oauth.reddit.com/r/wallstreetbets/new/?count=100&after=t3_ti0b84
t3_thukfw
25
https://oauth.reddit.com/r/teenagers/new/?count=25&after=t3_tjt2rv
t3_tjt2rv
50
https://oauth.reddit.com/r/teenagers/new/?count=50&after=t3_tjt2rv
t3_tjt2rv
75
https://oauth.reddit.com/r/teenagers/new/?count=75&after=t3_tjt2rv
t3_tjt2rv
100
https://oauth.reddit.com/r/teenagers/new/?count=100&after=t3_tjt2rv
t3_tjt2rv
25
https://oauth.reddit.com/r/copypasta/new/?count=25
t3_tqnftu
50
https://oauth.reddit.com/r/copypasta/new/?count=50
t3_tqnftu
75
https://oauth.reddit.com/r/copypasta/new/?count=75
t3_tqnftu
100
https://oauth.reddit.com/r/copypasta/new/?count=100
t3_tqnftu
25
https://oauth.reddit.com/r/genz/new/?count=25
t3_tosq84
50
https://oauth.re

In [10]:
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Convert list of dictionaries into pandas df
df = pd.DataFrame(my_list_of_dictionaries)
df.head()

,subreddit,text,title,url
0,wallstreetbets,,Accepting a share buyback without any shares =...,https://i.redd.it/k10jqp0pgto81.jpg
1,wallstreetbets,China Evergrande Group and its other units wer...,Evergrande Trading Suspended in Hong Kong; Ron...,https://www.reddit.com/r/wallstreetbets/commen...
2,wallstreetbets,"Ten months ago, u/robertisaak proposed, [here]...",South Park Investment Theory (deeper dive),https://www.reddit.com/r/wallstreetbets/commen...
3,wallstreetbets,"Checking flight radars, I have noticed a highl...",WSB AIR FORCE INTEL,https://www.reddit.com/r/wallstreetbets/commen...
4,wallstreetbets,,2 week 10x challenge. My starting position.,https://i.redd.it/ttwo0lw6pso81.jpg


In [11]:
df['subreddit'].value_counts()

wallstreetbets      100
copypasta           100
GenZ                100
unpopularopinion    100
Name: subreddit, dtype: int64

In [12]:
possible_labels = df.subreddit.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'wallstreetbets': 0, 'copypasta': 1, 'GenZ': 2, 'unpopularopinion': 3}

In [13]:
df['label'] = df.subreddit.replace(label_dict)

In [14]:
df.head()

,subreddit,text,title,url,label
0,wallstreetbets,,Accepting a share buyback without any shares =...,https://i.redd.it/k10jqp0pgto81.jpg,0
1,wallstreetbets,China Evergrande Group and its other units wer...,Evergrande Trading Suspended in Hong Kong; Ron...,https://www.reddit.com/r/wallstreetbets/commen...,0
2,wallstreetbets,"Ten months ago, u/robertisaak proposed, [here]...",South Park Investment Theory (deeper dive),https://www.reddit.com/r/wallstreetbets/commen...,0
3,wallstreetbets,"Checking flight radars, I have noticed a highl...",WSB AIR FORCE INTEL,https://www.reddit.com/r/wallstreetbets/commen...,0
4,wallstreetbets,,2 week 10x challenge. My starting position.,https://i.redd.it/ttwo0lw6pso81.jpg,0


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [16]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [17]:
df.groupby(['subreddit', 'label', 'data_type']).count()

text  title  url
subreddit        label data_type                  
GenZ             2     train        85     85   85
                       val          15     15   15
copypasta        1     train        85     85   85
                       val          15     15   15
unpopularopinion 3     train        85     85   85
                       val          15     15   15
wallstreetbets   0     train        85     85   85
                       val          15     15   15

In [18]:
df.head()

,subreddit,text,title,url,label,data_type
0,wallstreetbets,,Accepting a share buyback without any shares =...,https://i.redd.it/k10jqp0pgto81.jpg,0,train
1,wallstreetbets,China Evergrande Group and its other units wer...,Evergrande Trading Suspended in Hong Kong; Ron...,https://www.reddit.com/r/wallstreetbets/commen...,0,train
2,wallstreetbets,"Ten months ago, u/robertisaak proposed, [here]...",South Park Investment Theory (deeper dive),https://www.reddit.com/r/wallstreetbets/commen...,0,val
3,wallstreetbets,"Checking flight radars, I have noticed a highl...",WSB AIR FORCE INTEL,https://www.reddit.com/r/wallstreetbets/commen...,0,train
4,wallstreetbets,,2 week 10x challenge. My starting position.,https://i.redd.it/ttwo0lw6pso81.jpg,0,train


In [19]:
train_dataset = df[df.data_type == 'train']
val_dataset = df[df.data_type == 'val']

train_dataset = train_dataset.drop(['subreddit','title','url','data_type'], axis = 1)
val_dataset = val_dataset.drop(['subreddit','title','url','data_type'], axis = 1)


In [20]:
#have blanks
train_dataset.head()
val_dataset.head()

,text,label
2,"Ten months ago, u/robertisaak proposed, [here]...",0
14,,0
20,,0
21,**Read [rules](https://www.reddit.com/r/wallst...,0
23,Place your puts as Boeing stock is likely to t...,0


In [30]:
labels = tf.constant(train_dataset['label'])
features = tf.constant(train_dataset['text'])

train_dataset_tf = tf.data.Dataset.from_tensor_slices((features,labels))
#train_dataset_tf = tf.data.Dataset.from_tensor_slices(dict(train_dataset))
print(train_dataset_tf.element_spec)

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


In [22]:
labels_val = tf.constant(val_dataset['label'])
features_val = tf.constant(val_dataset['text'])

val_dataset_tf = tf.data.Dataset.from_tensor_slices((features_val,labels_val))
print(val_dataset_tf)

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int64)>


In [ ]:
#num_classes = 5 #number of classes, here is 10 (0,1,...,9)
#train_y = keras.utils.to_categorical(train_y, num_classes)
#test_y = keras.utils.to_categorical(test_y, num_classes)

In [31]:
#BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [32]:
train_dataset_tf = train_dataset_tf.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset_tf = val_dataset_tf.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [33]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset_tf.map(lambda text, labels: text))

In [34]:
#look at vocab
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'bill', 'the', 'to', 'and', 'of', 'a', 'is', 'i',
       'in', 'that', 'you', 'it', 'this', 'on', 'for', 'as', 'are', 'be'],
      dtype='<U15')

In [35]:
#make RNN model
batch_size = 64

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    #bidirectional expects 3D input
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=False)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [36]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              #optimizer=tf.keras.optimizers.Adam(1e-4),
              #metrics=['accuracy'])

In [38]:
history = model.fit(train_dataset_tf, epochs=10, batch_size=64,
                    validation_data=val_dataset_tf)

Epoch 1/10
6/6 [==============================] - 3s 344ms/step - loss: nan - accuracy: 0.2500 - val_loss: nan - val_accuracy: 0.2500
Epoch 2/10
6/6 [==============================] - 3s 348ms/step - loss: nan - accuracy: 0.2500 - val_loss: nan - val_accuracy: 0.2500
Epoch 3/10
6/6 [==============================] - 3s 358ms/step - loss: nan - accuracy: 0.2500 - val_loss: nan - val_accuracy: 0.2500
Epoch 4/10
6/6 [==============================] - 3s 365ms/step - loss: nan - accuracy: 0.2500 - val_loss: nan - val_accuracy: 0.2500
Epoch 5/10
6/6 [==============================] - 3s 361ms/step - loss: nan - accuracy: 0.2500 - val_loss: nan - val_accuracy: 0.2500
Epoch 6/10
6/6 [==============================] - 3s 365ms/step - loss: nan - accuracy: 0.2500 - val_loss: nan - val_accuracy: 0.2500
Epoch 7/10
6/6 [==============================] - 3s 368ms/step - loss: nan - accuracy: 0.2500 - val_loss: nan - val_accuracy: 0.2500
Epoch 8/10
6/6 [==============================] - 3s 375ms/ste

In [40]:
test_loss, test_acc = model.evaluate(val_dataset_tf)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

1/1 [==============================] - 0s 348ms/step - loss: nan - accuracy: 0.2500
Test Loss: nan
Test Accuracy: 0.25
